In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["THEANO_FLAGS"] = "OMP_NUM_THREADS=1, MKL_NUM_THREADS=1, OPENBLAS_NUM_THREADS=1"
os.environ["OMP_NUM_THREADS"] =  "1"
os.environ["MKL_NUM_THREADS"] =  "1" 
os.environ["OPENBLAS_NUM_THREADS"] =  "1" 

In [ ]:
### Initial imports
import logging
import numpy as np
import pandas as pd
import pymc3 as pm
import theano.tensor as T
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")

logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from epimodel.pymc3_models import cm_effect
from epimodel.pymc3_models.cm_effect.datapreprocessor import DataPreprocessor

%matplotlib inline

In [ ]:
dp = DataPreprocessor()
data = dp.preprocess_data("double_entry_final.csv", last_day="2020-05-30", schools_unis="single_features")

In [ ]:
cm_plot_style = [
#             ("\uf7f2", "tab:red"), # hospital symbol
            ("\uf963", "black"), # mask
            ("\uf0c0", "lightgrey"), # ppl
            ("\uf0c0", "grey"), # ppl
            ("\uf0c0", "black"), # ppl
            ("\uf07a", "tab:orange"), # shop 1
            ("\uf07a", "tab:red"), # shop2
            ("\uf549", "black"), # school
            ("\uf19d", "black"), # university
            ("\uf965", "black"), # home
            ("\uf072", "grey"), # plane1
            ("\uf072", "black"), # plane2
            ("\uf238", "black"), # train
            ("\uf1b9", "black"), # car
            ("\uf641", "black") # flyer
        ]

In [ ]:
data.summary_plot(cm_plot_style)

In [ ]:
data.mask_reopenings()

In [ ]:
pm.model_to_graphviz(model2).render("model-combined-final")
pm.model_to_graphviz(model2)

In [ ]:
with cm_effect.models.CMCombined_Final(data, cm_plot_style) as model2:
    model2.build_model()
    
    model2.trace = pm.sample(2000, tune=500, cores=4, chains=4, max_treedepth=12)

In [ ]:
pickle.dump(model2.trace, open("final_full.pkl", "wb"))

In [ ]:
model2.plot_effect()

In [ ]:
model2.plot_effect()

In [ ]:
np.savetxt("default_res.txt", model2.trace.CMReduction)